In [1]:
pip uninstall opencv-python
pip install opencv-python-headless
pip install opencv-python

SyntaxError: invalid syntax (825961016.py, line 1)

In [2]:
pip uninstall opencv-python opencv-python-headless opencv-contrib-python -y

Found existing installation: opencv-python 4.11.0.86
Uninstalling opencv-python-4.11.0.86:
  Successfully uninstalled opencv-python-4.11.0.86
Found existing installation: opencv-contrib-python 4.11.0.86
Uninstalling opencv-contrib-python-4.11.0.86:
  Successfully uninstalled opencv-contrib-python-4.11.0.86
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.Collecting opencv-python
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)



In [4]:
pip install opencv-contrib-python

  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyttsx3

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pyttsx3

In [4]:
engine = pyttsx3.init()
engine.setProperty('rate', 150)  # speed of speech (adjust if needed)

In [5]:
pip install pytesseract

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import easyocr
import pytesseract
from PIL import Image
import pyttsx3

 # Setup Text-to-Speech
engine = pyttsx3.init()
engine.setProperty('rate', 100)
# Canvas settings
canvas = np.ones((500, 800, 3), dtype="uint8") * 255
colors = [(0, 0, 0), (255, 0, 0), (0, 128, 0), (0, 0, 255)]
color_index = 0
cursor_pos = (0, 0)
drawing = False
last_point = None
tool = 'draw'  # draw or erase

# Drawing function
def draw(event, x, y, flags, param):
    global drawing, last_point, cursor_pos, canvas, tool
    cursor_pos = (x, y)

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_point = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        if last_point is not None:
            if tool == 'draw':
                cv2.line(canvas, last_point, (x, y), colors[color_index], 5)
            elif tool == 'erase':
                cv2.line(canvas, last_point, (x, y), (255, 255, 255), 20)
            last_point = (x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        last_point = None

# Setup window and callback
cv2.namedWindow("Draw")
cv2.setMouseCallback("Draw", draw)

while True:
    display = canvas.copy()

    # Guides
    cv2.line(display, (50, 400), (750, 400), (200, 200, 200), 1)
    cv2.rectangle(display, (50, 100), (750, 600), (220, 220, 220), 1)

    # Cursor
    cv2.circle(display, cursor_pos, 2, (50, 50, 50), -1)

    # Info
    cv2.putText(display, f"Mode: {tool.upper()}", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1)
    if tool == 'draw':
        cv2.putText(display, f"Color: {colors[color_index]}", (10, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[color_index], 1)
    cv2.putText(display, f"Cursor: {cursor_pos}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Help text
    cv2.putText(display, "D: Draw | E: Erase | N: Next Color | C: Clear | S: Save/Extract | ESC: Exit",
                (10, 480), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (100, 100, 100), 1)

    cv2.imshow("Draw", display)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('d'):
        tool = 'draw'
    elif key == ord('e'):
        tool = 'erase'
    elif key == ord('c'):
        canvas[:] = 255
    elif key == ord('n'):
        color_index = (color_index + 1) % len(colors)
    elif key == ord('s'):
        cv2.imwrite("drawing_output.png", canvas)
        print("Saved as drawing_output.png")

        gray = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        coords = cv2.findNonZero(thresh)
        if coords is not None:
            x, y, w, h = cv2.boundingRect(coords)
            cropped = canvas[y:y+h, x:x+w]

            cropped_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
            _, prepped = cv2.threshold(cropped_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

            kernel = np.ones((2, 2), np.uint8)
            dilated = cv2.dilate(prepped, kernel, iterations=1)

            cv2.imshow("Extracted Drawing", cropped)

            # EasyOCR
            reader = easyocr.Reader(['en'])
            result = reader.readtext(dilated)

            print("\n🧠 EasyOCR Output:")
            spoken_text = ""
            for (_, text, conf) in result:
                print(f"📝 Detected: {text} (Confidence: {conf:.2f})")
                spoken_text += text + " "

            # Fallback to Tesseract
            if not spoken_text.strip():
                dilated_rgb = cv2.cvtColor(dilated, cv2.COLOR_GRAY2RGB)
                pil_img = Image.fromarray(dilated_rgb)
                spoken_text = pytesseract.image_to_string(pil_img, lang='eng').strip()
                print("\n🔄 Tesseract Backup OCR:")
                print(spoken_text)
        
            # Text-to-Speech
            if spoken_text.strip():
                print(f"\n🔊 Speaking: {spoken_text}")
                engine.say(spoken_text)
                engine.runAndWait()
            else:
                print("❌ No text found to speak.")
        else:
            print("❌ No handwriting detected on canvas!")
    elif key == 27:
        break

cv2.destroyAllWindows()


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Saved as drawing_output.png

🧠 EasyOCR Output:
📝 Detected: PAPA (Confidence: 0.86)

🔊 Speaking: PAPA 


In [2]:
voices = engine.getProperty('voices') #this will give the list of available voices
engine.setProperty('voice', voices[0].id)